In [1]:
import gp

In [2]:
# Create a GenePattern server proxy instance
gpserver = gp.GPServer('https://genepattern.broadinstitute.org/gp','Simpli', 'Simpli')

In [3]:
task_list = gpserver.get_task_list()
for t in task_list:
    print(t.get_name())

ABSOLUTE
ABSOLUTE.review
ABSOLUTE.summarize
Acgh2Tab
AddFCSEventIndex
AddFCSParameter
AddNoiseToFCS
AffySTExpressionFileCreator
ApplyGatingML
ARACNE
AreaChange
Arff2Gct
ATARiS
AuDIT
BedToGtf
Beroukhim.Getz.2007.PNAS.Glioma.GISTIC
BlastTrainTest
BlastXValidation
Bowtie.aligner
Bowtie.indexer
BWA.aln
BWA.bwasw
BWA.indexer
CaArray2ImportViewer
CART
CARTXValidation
CBS
ChIPSeq.CreateHeatmap
ChIPSeq.CreatePlots
ChIPSeq.ExtractSignal
ClassNeighbors
ClsFileCreator
CNToXCN
CoClustering
CollapseDataset
ComBat
CombineOdf
ComparativeMarkerSelection
ComparativeMarkerSelectionViewer
CompareSpectra
CompensateFCS
ConcatenateFilelist
ConsensusClustering
ConstellationMap
ConvertLineEndings
ConvertToMAGEML
ConvertToMAGETAB
COPA
CopyNumberDivideByNormals
CopyNumberInferencePipeline
CopyNumberNoise
copyOfReorderGCT
CoxRegression
CreateSymlinks
CsvToFcs
Cuffcompare
Cuffdiff
Cufflinks
CufflinksCuffmergePipeline
Cuffmerge
CummeRbund.GeneSetReport
CummeRbund.QcReport
CummeRbund.SelectedConditionsReport
CummeR

In [6]:
t = gp.GPTask(gpserver, 'PreprocessDataset')
t.param_load()
ps = t.get_parameters()
for p in ps:
    print('{} = {}'.format(p.get_name(), p.get_default_value()))

input.filename = None
threshold.and.filter = 1
floor = 20
ceiling = 20000
min.fold.change = 3
min.delta = 100
num.outliers.to.exclude = 0
row.normalization = 0
row.sampling.rate = 1
threshold.for.removing.rows = None
number.of.columns.above.threshold = None
log2.transform = 0
output.file.format = 3
output.file = <input.filename_basename>.preprocessed


In [32]:
js = t.make_job_spec()

In [37]:
for p in t.get_parameters():
    if p.get_default_value():
        print(js.set_parameter(p.get_name(), p.get_default_value()))

None
None
None
None
None
None
None
None
None
